In [6]:
#requirements
import pandas as pd

#for tokenizer
import morfessor
import math

# Repression Prediction vis LLM
The linear regression didn't perform very well. Lets try this again but with a LLM trained to the sequence data. In order to have peak performance, we need to make a specific tokenizer for this model. The tokenizer should be able to identify key motifs in the sequence. 

In [5]:
#train morfessor model to tokenize words
io = morfessor.MorfessorIO()
train_data = list(io.read_corpus_file('data/morphemes.txt'))
model = morfessor.BaselineModel()

def sigmoid(x):
   return 5/(1+(math.e**(-(x-3.188))))

model.load_data(train_data, count_modifier=sigmoid)
model.train_batch()

NameError: name 'morfessor' is not defined

In [6]:
def tokenize_by_word(seq, model):
    return " ".join(model.viterbi_segment(seq, addcount=1,maxlen=7)[0])

In [32]:
def tokenize_by_sliding_window(seq, rbp_dict, treshhold=3.32):
    seq_string = ""
    for i in range(len(seq)-6):
        subseq = seq[i:i+7]
        if subseq in rbp_dict and rbp_dict[subseq]['Mean_RBP'] > treshhold:    
            seq_string = seq_string+subseq + " " 
    return seq_string

In [22]:
#format data by 'morfessor word tokens'
def format_data_word_tokens(model):
    exp_data = pd.read_csv("data/Rabani_expression_A+.csv")
    seq_data = pd.read_csv("data/Rabani_oligos.csv")

    #generate ER
    er_list = []
    token_list = []
    count = 0
    for ind,row in exp_data.iterrows():
        seq_id = exp_data.loc[ind, 'id'] 
        seq = seq_data.loc[seq_data['id'] == seq_id, 'Sequence'].values
        #see if sequence is in seq_data
        if seq.size!=0:
            er = (exp_data.loc[ind, '6h'] - exp_data.loc[ind, '1h'])/2
            er_list.append(er)
            token_list.append(tokenize_by_word(seq[0],model))
            if count%400==0:
                print("")  
            elif count%5==0: 
                print(".", end ="")
        count+=1
    #print token_list to text file for tokenizer training
    with open("data/token_data_word_token.txt", "w") as txt_file:
        for line in token_list:
            txt_file.write(line+"\n")
    df = pd.DataFrame({'Token':token_list,'Expression Rate':er_list})
    df.to_csv("data/seq_er_word_token.csv",index=False)

In [46]:
#format data by 'sliding window with mean rbp threshold'
def format_data_slideing_window():
    exp_data = pd.read_csv("data/Rabani_expression_A+.csv")
    seq_data = pd.read_csv("data/Rabani_oligos.csv")
    rbp_dict = pd.read_csv("data/ZF_7N_PCBP2_vs_Adar-1.csv")[['Motif', 'Mean_RBP']].set_index('Motif').T.to_dict()

    #generate ER
    er_list = []
    token_list = []
    count = 0
    for ind,row in exp_data.iterrows():
        seq_id = exp_data.loc[ind, 'id'] 
        seq = seq_data.loc[seq_data['id'] == seq_id, 'Sequence'].values
        #see if sequence is in seq_data
        if seq.size!=0:
            tokens = tokenize_by_sliding_window(seq[0], rbp_dict)
            if len(tokens) > 0:
                er = (exp_data.loc[ind, '6h'] - exp_data.loc[ind, '1h'])/2
                er_list.append(er)
                token_list.append(tokens)
            else:
                print("None Token found:")
            if count%5000==0:
                print("")  
            elif count%200==0: 
                print(".", end ="")
        count+=1
    #print token_list to text file for tokenizer training
    with open("data/token_data_sliding_window.txt", "w") as txt_file:
        for line in token_list:
            txt_file.write(line+"\n")
    df = pd.DataFrame({'Token':token_list,'Expression Rate':er_list})
    df.to_csv("data/seq_er_sliding_window.csv",index=False)

In [47]:
format_data_slideing_window()

........................
........................
........................
........................
........................
........................
........................
........................
........................
........................
........................
........................
........................
........................
........................
..............None Token found:
..........
........................
............None Token found:
..None Token found:
..........
